In [1]:
!sudo pip install -q transformers --upgrade

In [6]:
!sudo -H pip install auto-gptq --no-cache-dir

In [3]:
!sudo -H pip install --upgrade optimum

     |████████████████████████████████| 409kB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 18.0MB/s eta 0:00:01
     |████████████████████████████████| 92kB 14.5MB/s eta 0:00:01


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

CUDA extension not installed.
CUDA extension not installed.


In [5]:
model_name = "/data/mistral/query-to-mql/exp-10/nov-20/merged-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
quantizer = GPTQQuantizer(bits=4, dataset="c4", model_seqlen = 8192)

In [8]:
quantized_model = quantizer.quantize_model(model, tokenizer)

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

AttributeError: 'MistralModel' object has no attribute 'decoder'

In [ ]:
# above step gets stuck at generating train split

In [17]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [18]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [ ]:
model_inputs = tokenizer(input_text, return_tensors='pt').to(torch_device)
greedy_output = model.generate(**model_inputs, max_new_tokens=1024) #speedup obtained

In [19]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model.generate(input_ids=_inputs, max_length= 1700, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output

In [21]:
import time

In [23]:
del model

In [7]:
!sudo pip install vllm

     |████████████████████████████████| 29.0MB 8.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.3MB/s  eta 0:00:01
     |████████████████████████████████| 3.2MB 102.7MB/s eta 0:00:01
     |███████████████▎                | 319.0MB 111.4MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |█████████████████████████████   | 609.5MB 177.6MB/s eta 0:00:01

     |█████████████████████████████▌  | 617.6MB 177.6MB/s eta 0:00:01

     |█████████████████████████████▉  | 625.8MB 177.6MB/s eta 0:00:01

...

     |████████████████████████████████| 731.5MB 180.6MB/s eta 0:00:01

     |████████████████████████████████| 731.7MB 8.8kB/s s eta 0:00:01
     |████████████████████████████████| 14.1MB 38kB/s s eta 0:00:01
     |████████████████████████████████| 124.2MB 23kB/s s eta 0:00:01
     |███████████████████████         | 295.3MB 143.3MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 410.6MB 12kB/s 
     |████████████████████████████████| 196.0MB 13kB/s /s eta 0:00:01
     |████████████████████████████████| 829kB 105.9MB/s eta 0:00:01
     |████████████████████████████████| 23.7MB 100.2MB/s eta 0:00:01
     |████████████████████████████████| 56.5MB 105kB/s  eta 0:00:01
     |████████████████████████████████| 102kB 10.1MB/s eta 0:00:01
     |████████████████████████████████| 89.2MB 66kB/s s eta 0:00:01
     |████████████████████████████████| 209.8MB 27kB/s /s eta 0:00:01
     |████████████████████████████████| 121.6MB 16kB/s s eta 0:00:01
     |████████████████████████████████| 542kB 86.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 8.0MB/s  eta 0:00:01
     |████████████████████████████████| 71kB 6.4MB/s  eta 0:00:01
     |████████████████████████████████| 20.6MB 102.2MB/s eta 0:00:01
ERROR: torchvision 0.15.2+cpu has requirement torch==2.0.1, but you'll have torch 2.1.1 which is incompatible.
ERROR: torchaudi

  Found existing installation: anyio 3.7.0
    Uninstalling anyio-3.7.0:
      Successfully uninstalled anyio-3.7.0
  Found existing installation: torch 2.0.1+cpu
    Uninstalling torch-2.0.1+cpu:
      Successfully uninstalled torch-2.0.1+cpu


In [8]:
from vllm import LLM, SamplingParams

ImportError: libcudart.so.12: cannot open shared object file: No such file or directory

In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [ ]:
llm = LLM(model="facebook/opt-125m")

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [22]:
%%time
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'
print('user query: ', user_query)
print('-'*100)
start = time.time()
output, raw = predict_template_query_v1(user_query=user_query)
print('total time taken: ', time.time()-start)

user query:  sales in nov 2020 to may 2021 vs Q3 2021
----------------------------------------------------------------------------------------------------


AttributeError: 'MistralForCausalLM' object has no attribute 'jforward_multilevel'

In [ ]:
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [17]:
%%time
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'
print('user query: ', user_query)
print('-'*100)
start = time.time()
output, raw = predict_template_query_v1(user_query=user_query)
print('total time taken: ', time.time()-start)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales in nov 2020 to may 2021 vs Q3 2021
----------------------------------------------------------------------------------------------------
total time taken:  68.28918886184692
{'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}, 'DATE VARIABLE': {'nov 2020 to may 2021': [{'ENTITY': 'Order Date', 'DATE RANGE': '01/11/2020 - 31/05/2021', 'CONVERTED TIME ELEMENT': 'November 2020 to May 2021'}], 'Q3 2021': [{'ENTITY': 'Order Date', 'DATE RANGE': '01/07/2021 - 30/09/2021', 'CONVERTED TIME ELEMENT': 'Q3 2021'}]}}
----------------------------------------------------------------------------------------------------


IndexError: list index out of range

In [4]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


# Load base model

model_name = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


In [ ]:
quantizer.quantize(
    input_data=dataset["train"],
    valid_data=dataset["validation"],
)